### Create & Load Documents for LLM use

In [ ]:
"""
LLM 과 Retriever를 사용하기 앞서 input 문서들을 LLM 및 Retriever가 받아들일 수 있는
포맷으로 바꿔 주어야 함. 그러기 위해서는 langchain 라이브러리에 있는 'Document' 사용해야함.

'from langchain.schema import Document'

Document내의 속성으로는:
    1. 'page_content' : 내용
    2. 'metadata : 딕셔너리 형태의 Document의 귀속하는 정보

를 나타내면 됨.


기본적으로 LangChain 라이브러리는 직접 Document를 작성하지 않아도 사용할 수 있는 메소드가
존재하며, pdf, excel, text, docx, url 등등의 파일을 함수로 통해 Document format으로 쉽게
가져올 수 있음. 

(Document로 직접 구축하지 않아도 되지만, metadata를 잘 추출하였는지의 여부를 확인하길 권장)

ex)
    1. CSVLoader
    2. UnstructuredHTMLLoader
    3. JSONLoader
    4. UnstructuredMarkdownLoader
    5. PyPDFLoader
    6. TextLoader
    .
    .
    etc...

references:
URL : https://python.langchain.com/v0.1/docs/modules/data_connection/document_loaders/

"""

In [14]:
## [본문 및 메타데이터 작성]

# Libraries
from langchain.schema import Document
from langchain_community.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Documents Create
Docs = [
    Document(
        page_content="""서울시가 송파구 잠실동, 강남구 삼성·대치·청담동 일대를 토지거래허가구역으로 재지정했다.
        13일 서울시는 국제교류복합지구와 인근 지역의 토지거래허가구역을 재지정 심의·의결했다고 밝혔다. 앞서 5일 도시계획위원회에서 결정이 '보류'됐지만 결국 연장됐다.
        서울시 관계자는 "최근 서울은 아파트 위주로 회복세가 나타나고 특히 강남 3구의 회복률이 높다"며 "이달 들어 서울 전역의 아파트 매매가격이 상승 전환한 만큼 규제를 풀면 불안해질 소지가 있다"고 말했다.
        다만, 위원회는 토지거래허가구역 지정 효과에 대한 실효성 논란이 있는 만큼 근본적인 해법을 찾겠단 방침이다. 연내 도시계획위원회에 안건을 상정해 재검토하기로 했다.
        시 관계자는 "오는 7~8월께 전문가들과 토론회를 열 예정"이라며 "데이터에 기반해 제도적으로 효과가 있는지를 종합적으로 다룰 것"이라고 설명했다.
        토지거래허가구역으로 묶이면 기준 이상의 주택·상가·토지 등을 거래할 때 시·군·구청장의 허가를 받아야 한다. 주거 목적으로 매입할 경우 2년간 실거주 의무가 있어 전세를 끼고 주택을 매입하는 '갭투자'가 불가능하다.
        """,
        metadata={"title":"서울 잠실·삼성·청담·대치, 토지거래허가구역 1년 더 연장", "date": "2024/06/13", "category":"경제"}
    ),
    Document(
        page_content="""미국을 방문한 이재용 삼성전자 회장이 메타·아마존·퀄컴 등 글로벌 빅테크 수장들을 잇달아 만나 인공지능(AI)과 반도체·스마트폰·통신·증강현실 등 광범위한 사업 협력을 논의했다.
        13일 삼성전자에 따르면 이재용 회장은 지난 11일(현지시간) 미국 캘리포니아주 팔로알토에 위치한 마크 저커버그 메타 최고경영자(CEO)의 자택에서 단독 미팅을 가졌다. 지난 2월 저커버그 CEO 방한 이후 4개월 만에 다시 만난 것으로, 두 사람은 AI와 가상현실·증강현실 등에서의 협력 방안을 논의했다.
        메타는 삼성전자 파운드리(반도체 위탁생산) 부문의 잠재 고객사다. 자체 거대언어모델(LLM) ‘라마’를 훈련시키는 용도로 지난해 AI 반도체 ‘MTIA’를 처음 선보였다. 대만 파운드리 TSMC가 MTIA를 생산하고 있는데 업계에서는 메타가 이 물량 일부를 삼성전자로 돌릴 가능성이 있다고 본다.
        이튿날인 12일 이 회장은 시애틀 아마존 본사에서 앤디 재시 아마존 CEO를 만났다. 두 사람은 생성형 AI와 클라우드 컴퓨팅 등에 대한 전망을 공유하며 추가 협력을 논의했다. 아마존은 계열사 아마존웹서비스(AWS)의 클라우드 사업에 AI 서비스를 접목하기 위해 공을 들이고 있다. 자체 AI 반도체 ‘트레이니움’을 만들어 쓰고 있으며, 여기에는 삼성전자의 고대역폭메모리(HBM)가 탑재된다.
        이 회장은 지난 10일에는 크리스티아누 아몽 퀄컴 CEO도 만나 AI 반도체와 차세대 통신칩 등의 협력을 논의했다. 이 회장은 퀄컴뿐만 아니라 다른 미국의 팹리스(생산라인 없는 반도체 설계사) 기업들과도 만나 파운드리 사업을 논의한 것으로 전해졌다.
        삼성전자는 최근 반도체 사업에서 과거 ‘초격차’로 대변되던 경쟁력을 잃었다는 안팎의 평가를 받고 있다. 이 회장의 미국 출장은 메모리·파운드리 사업에서 돌파구를 마련하려는 행보로 해석된다. 지난달 31일 출국해 2주간의 일정을 마무리한 이 회장은 임원들에게 “삼성의 강점을 살려 삼성답게 미래를 개척하자”고 말했다. 삼성전자는 이달 말 주요 경영진이 참여하는 ‘글로벌 전략회의’를 열 예정이다.
        """,
        metadata={"title":"4개월 만에 또 저커버그 만난 이재용 AI 분야 협력", "date": "2024/06/13", "category":"IT"}
    )
]

# Embedding
model_name = "jhgan/ko-sbert-nli"
encode_kwargs = {"normalize_embeddings":True}
ko_embedding = HuggingFaceEmbeddings(
    model_name = model_name,
    encode_kwargs = encode_kwargs)

# Create Child Documents - text splitter - 자식 문서 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
splits = text_splitter.split_documents(Docs)

# Vectorstore
vectorstore = Chroma.from_documents(
                                documents=splits,
                                embedding=ko_embedding
                                )

In [15]:
## [LLM 및 Retriever 정의]

# Library
from langchain.chains.query_constructor.base import AttributeInfo
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain.chat_models import ChatOllama

# document_content_description = "뉴스기사"
llm = ChatOllama(model="wizardlm2")
retriever = MultiQueryRetriever.from_llm(
    llm=llm,
    retriever=vectorstore.as_retriever(),
    )


In [20]:
## [Chain 연결]

# Libraries
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# prompt engineering을 통해 더 나은 성능을 이끌어 낼 수 있음.
final_template = """Answer the question based ONLY on the following context:
                    {context}
                    Question: {question}
                    """

# Chain 연결
prompt = PromptTemplate.from_template(final_template)
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [21]:
# 질문 및 결과 확인 :)
ans = chain.invoke("이재용이 만난 사람들은 누구야?")
print(ans)

이재용 전무(현재 임원)가 4개월 만에 다시 저커버그의 지적 재산권을 보유하고 있는 제프 빈(Jeff Bezos)과 만난 사람들에 대한 정보는 공식 발표나 공개된 기록에서 확인할 수 있습니다. 위의 설명에 따르면, 이재용 전무는 제프 빈과의 미팅을 통해 삼성전자의 AI 분야에서의 협력 가능성을 모색하고 있으며, 이는 삼성전자의 글로벌 전략회의에서도 그 행보를 계속 추진하고 있음을 보여줍니다.

제프 빈은 저커버그의 CEO인 동시에 그 회사의 주요 임상자로, 이재용과의 만남은 삼성전자와 아마존 간의 협력이나 경쟁 관련 내용을 다룰 수 있는 기회로 해석됩니다. 그러나 구체적인 참석자 목록이나 미팅의 세부 내용은 공개된 정보가 없거나 제한될 수 있으므로, 이 외에도 삼성전자와 저커버그 간의 다른 임원들이나 관련 당사자들과의 만남이 포함될 수 있습니다.

위의 내용에서는 제프 빈 외에도 삼성전자의 메모리와 파운드리 사업에서의 돌파구를 마련하기 위한 미국 출장과 글로벌 전략회의에 대한 언급이 있습니다. 따라서, 이재용 전무가 만난 사람들은 제ф 빈을 포함한 저커버그의 임원, 삼성전자의 프로젝트 담당자, 그리고 글로벌 전략회의에 참여하는 삼성전자의 주요 경영진이 될 수 있습니다.
